# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, we will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage us in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |


In [24]:
-- top_five_products_each_category 
With r AS (Select category, product_name , Round(Sum(o.profit)::numeric, 2) AS total_profit,ROUND(Sum(o.sales)::numeric , 2) AS total_sales ,  RANK() OVER(PARTITION BY p.category ORDER BY SUM(o.sales) DESC) AS product_rank
From products AS p
Left Join orders AS o
ON p.product_id = o.product_id		   
Group BY category, product_name)

Select * 
From r
Where product_rank < 6



,category,product_name,total_profit,total_sales,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",5997.25,58193.48,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",4925.80,51449.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",10427.33,50121.52,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",7154.28,41923.53,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",5562.35,40585.13,5
5,Office Supplies,"Eldon File Cart, Single Width",5571.26,39873.23,1
6,Office Supplies,"Hoover Stove, White",-2180.63,32842.60,2
7,Office Supplies,"Hoover Stove, Red",11651.68,32644.13,3
8,Office Supplies,"Rogers File Cart, Single Width",2368.82,29558.82,4
9,Office Supplies,"Smead Lockers, Industrial",3630.44,28991.66,5


In [25]:
-- salesperson_market_sales_details
SELECT ppl.person,
	ord.market,
	CASE WHEN ord.sales >= 0 AND ord.sales < 100	THEN '0-100'
		WHEN ord.sales >= 100 AND ord.sales < 500 THEN '100-500'
		WHEN ord.sales >= 500 THEN '500+'
		END AS sales_bin,
		COUNT(DISTINCT ord.order_id) AS order_counts,
		SUM(CASE WHEN ret.returned IS NULL THEN 0 ELSE 1 END) AS orders_returned,
		SUM(ord.sales) AS total_sales,
		SUM(CASE WHEN ret.returned IS NULL THEN 0 ELSE ord.sales END) AS returned_sales
FROM orders as ord
INNER JOIN people AS ppl
	ON ord.region = ppl.region
LEFT JOIN returned_orders AS ret
	ON ord.order_id = ret.order_id AND ord.market = ret.market
GROUP BY sales_bin, ord.market, ppl.person
ORDER BY ppl.person, ord.market, sales_bin;


,person,market,sales_bin,order_counts,orders_returned,total_sales,returned_sales
0,Alejandro Ballentine,APAC,0-100,995,69,6.654857e+04,3440.56500
1,Alejandro Ballentine,APAC,100-500,797,59,2.636462e+05,15190.23870
2,Alejandro Ballentine,APAC,500+,428,25,5.542284e+05,25047.59400
3,Anna Andreadi,EU,0-100,1662,159,1.239398e+05,7951.05900
4,Anna Andreadi,EU,100-500,1524,162,5.299971e+05,37894.53000
5,Anna Andreadi,EU,500+,818,61,1.066616e+06,61685.20950
6,Anna Andreadi,LATAM,0-100,1080,77,7.041208e+04,2964.17896
7,Anna Andreadi,LATAM,100-500,734,43,2.179294e+05,10151.38944
8,Anna Andreadi,LATAM,500+,286,9,3.121686e+05,9244.36624
9,Anna Andreadi,US,0-100,904,0,4.254738e+04,0.00000


In [26]:
-- impute_missing_values
WITH missing AS (
	SELECT product_id,
		discount, 
		market,
		region,
		sales,
		quantity
	FROM orders 
	WHERE quantity IS NULL
), 

unit_prices AS (SELECT o.product_id,
	CAST(o.sales / o.quantity AS NUMERIC) AS unit_price
FROM orders o
RIGHT JOIN missing AS m 
	ON o.product_id = m.product_id
	AND o.discount = m.discount
WHERE o.quantity IS NOT NULL
)

SELECT DISTINCT m.*,
	ROUND(CAST(m.sales AS NUMERIC) / up.unit_price,0) AS calculated_quantity
FROM missing AS m
INNER JOIN unit_prices AS up
	ON m.product_id = up.product_id;

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4
1,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,2
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3
3,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2
4,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,4
